In [2]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, LSTM


data = pd.read_csv("../data/train.txt", sep=';')
data.columns = ["Text", "Emotions"]
data.head()

,Text,Emotions
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness


As this is a problem of natural language processing, I’ll start by tokenizing the data:

In [3]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

Now we need to pad the sequences to the same length to feed them into a neural network. Here’s how we can pad the sequences of the texts to have the same length:

In [4]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

Now I’ll use the label encoder method to convert the classes from strings to a numerical representation:

In [5]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

We are now going to One-hot encode the labels. One hot encoding refers to the transformation of categorical labels into a binary representation where each label is represented as a vector of all zeros except a single 1. This is necessary because machine learning algorithms work with numerical data. So here is how we can One-hot encode the labels:

In [6]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

## Text Emotions Classification Model
Now we will split the data into training and test sets:

In [7]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, 
                                                one_hot_labels, 
                                                test_size=0.2)

Now let’s define a neural network architecture for our classification problem and use it to train a model to classify emotions:

In [8]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=132, input_length=max_length))
model.add(LSTM(units=64))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=5, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/5


2024-04-24 23:52:13.579119: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


400/400 [==============================] - 18s 42ms/step - loss: 1.2154 - accuracy: 0.5430 - val_loss: 0.6482 - val_accuracy: 0.7897
Epoch 2/5
400/400 [==============================] - 17s 43ms/step - loss: 0.3267 - accuracy: 0.9000 - val_loss: 0.2800 - val_accuracy: 0.9041
Epoch 3/5
400/400 [==============================] - 18s 44ms/step - loss: 0.1238 - accuracy: 0.9612 - val_loss: 0.3014 - val_accuracy: 0.9000
Epoch 4/5
400/400 [==============================] - 18s 45ms/step - loss: 0.0727 - accuracy: 0.9765 - val_loss: 0.2938 - val_accuracy: 0.9066
Epoch 5/5
400/400 [==============================] - 17s 42ms/step - loss: 0.0491 - accuracy: 0.9845 - val_loss: 0.3099 - val_accuracy: 0.9097


Now let’s take a sentence as an input text and see how the model performs:

In [9]:
model_architecture = model.to_json()
with open("../data/model_architecture.json", "w") as json_file:
    json_file.write(model_architecture)

model.save_weights("../data/model_weights.h5")

In [64]:
input_text = "I have mixed feelings"
input_text.lower()

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)
print(prediction)

1/1 [==============================] - 0s 14ms/step
['joy']
[[3.3702879e-04 3.7899020e-04 9.9682981e-01 6.7955483e-04 1.5563468e-03
  2.1814996e-04]]


So this is how you can use Machine Learning for the task of text emotion classification using the Python programming language.